In [248]:
import astropy 
from astropy.table import Table, Column, join
import numpy as np
from astropy.coordinates import SkyCoord
from astropy import units as u
spatial_err = 0.3/3600*u.degree

In [213]:
#threshold is in units of spatial_err (typically 0.3 arcsec)
#we want to merge mag_24 in source into target
def coord_merge(target, source, threshold):
    tc = SkyCoord(ra=target['ra'], dec=target['dec'])
    sc = SkyCoord(ra=source['ra'], dec=source['dec'])
    idx, d2d, d3d = sc.match_to_catalog_sky(tc)
    #idx is index into tc (target) coordinates that are closest objects to each of the coordinates in sc (source)
    #There are as many objects in idx as in source
    for i in range(0, len(idx)):
        if d2d[i] < threshold * spatial_err:
            target['mag_24'][idx[i]] = source['mag_24'][i]
            print 'merged'       
    return 0

In [240]:
#compute the minimum separation between two catalogs
#for each object in source, find the object in target that is closest
#returns min(dist(obj1, obj2)), obj1 in catalog 1, obj2 in catalog 2
#The order of the arguments do not change the result, but using the smaller catalog as the second argument will 
#the function run faster
def cat_dist(target, source):
    tc = SkyCoord(ra=target['ra'], dec=target['dec'])
    sc = SkyCoord(ra=source['ra'], dec=source['dec'])
    idx, d2d, d3d = sc.match_to_catalog_sky(tc)
    return min(d2d)

In [245]:
#merge mag_24 in source into target catalog
def desig_merge(target, source):
    return join(target, source, keys='desig', join_type = 'left')

In [202]:
g = Table.read('/home/kecai/w49/glimpse_demo.tbl', format='ascii')
m = Table.read('/home/kecai/w49/mipsgal_demo.tbl', format='ascii')

In [252]:
m.rename_column('desig', 'desigation')
m.rename_column('mag_24', 'mag24')
m.rename_column('sigma_mag_24', 'd24m')

In [247]:
#strip the 8-character prefix in glimpse designation
for i in range(0, len(g['desig'])):
    g['desig'][i] = g['desig'][i][8:]

In [191]:
m_w_desig = m[m['desig'].mask==False]['desig', 'mag_24'] 

In [221]:
m_wo_desig = m[m['desig'].mask==True]['ra', 'dec', 'mag_24']

In [210]:
merged = join(g, m_w_desig, keys='desig', join_type='left')

In [243]:
cat_dist(g, m_wo_desig)

<Angle 0.0006050000000005055 deg>

In [234]:
min(d2d)

<Angle 5.623651035797921e-06 deg>

In [212]:
coord_merge(merged, m_wo_desig, 3)

[39975 31019 33362 22308 13415 37312 37178 27772 34661 26301  1526  2144
  2273 11174 12213   357 40290 41355 34873 38852]


0

In [169]:
gc = SkyCoord(ra=g['ra'], dec=g['dec'])  

In [170]:
mc = SkyCoord(ra=m_wo_desig['ra'], dec=m_wo_desig['dec'])

In [153]:
idx, d2d, d3d = mc.match_to_catalog_sky(gc)  

In [180]:
len(idx)

20